In [ ]:
import csv
import json
import numpy as np
import os.path
import pandas as pd
import re
import requests
import time
from datetime import datetime
from pytz import timezone

In [ ]:
BICING_URL = "http://wservice.viabicing.cat/v2/stations"
OUTPUT_PATH = "output"
STATIONS_FILE = "output/stations.csv"
SECONDS_UPDATE = 40

In [ ]:
def FetchBicingData(url = BICING_URL):
    r = requests.get(url)
    bicingJson = r.json()
    updateTime = bicingJson["updateTime"]
    bicingDf = pd.DataFrame.from_dict(bicingJson["stations"])
    bicingDf["nearbyStations"] = bicingDf["nearbyStations"].apply(lambda x: re.sub(", ", " - ", x))
    bicingDf["updateTime"] = ConvertTimestampToUtcString(updateTime)
    return(bicingDf)

def ConvertTimestampToUtcString(timestamp):
    timestampLocal = datetime.fromtimestamp(timestamp)
    timestampUtc = timestampLocal.astimezone(timezone("UTC"))
    timestampString = timestampUtc.strftime("%Y-%m-%d %H:%M:%S %Z%z")
    return(timestampString)

def CreateStationsFile(file = STATIONS_FILE):
    fieldsList = ["internalId", "id", "type", "streetName", "streetNumber", "latitude",
                   "longitude", "altitude"]
    with open(file, 'w', newline = '') as myfile:
        wr = csv.writer(myfile, delimiter = ";")
        wr.writerow(fieldsList)
#    print("Stations file created.")

def CreateDataFile(internalId, outputPath = OUTPUT_PATH):
    internalId = int(internalId)
    fieldsList = ["updateTime", "status", "bikes", "slots", "nearbyStations"]
    file = outputPath + "/" + str(internalId).zfill(5) + ".csv"
    with open(file, 'w', newline = '') as myfile:
        wr = csv.writer(myfile, delimiter = ";")
        wr.writerow(fieldsList)
#    print("Data file for station " + str(internalId) + " created.")

def ReadStationsFile(file = STATIONS_FILE):
    converters = {"id": str, "type": str, "streetName": str, "streetNumber": str,
                  "latitude": str, "longitude": str, "altitude": str}
    if os.path.isfile(file) == False:
        CreateStationsFile(file)
    stations = pd.read_csv(file, delimiter = ";", encoding = "latin-1", converters = converters)
    return(stations)

def UpdateStationsFile(internalId, row, file = STATIONS_FILE):
    internalId = int(internalId)
    values_list = [internalId, row["id"], row["type"], row["streetName"], row["streetNumber"], row["latitude"],
                   row["longitude"], row["altitude"]]
    with open(file, 'a', newline = '') as myfile:
        wr = csv.writer(myfile, delimiter = ";", quoting=csv.QUOTE_ALL)
        wr.writerow(values_list)
#    print("Stations file updated with station " + str(internalId) + ".")

def UpdateDataFile(internalId, row, outputPath = OUTPUT_PATH):
    internalId = int(internalId)
    values_list = [row["updateTime"], row["status"], row["bikes"], row["slots"], row["nearbyStations"]]
    file = outputPath + "/" + str(internalId).zfill(5) + ".csv"
    with open(file, 'a', newline = '') as myfile:
        wr = csv.writer(myfile, delimiter = ";")
        wr.writerow(values_list)
#    print("Data for station " + str(internalId) + " updated.")

def SaveBicingData(stationsFile = STATIONS_FILE, outputPath = OUTPUT_PATH):
    stations = ReadStationsFile(stationsFile)
    bicing = FetchBicingData()
    bicingStations = pd.merge(bicing, stations[["internalId", "id", "type", "latitude", "longitude", "altitude"]],
                              on = ["id", "type", "latitude", "longitude", "altitude"],
                              how = "left")
    if np.all(bicingStations["internalId"].isna()):
        nextStation = 1
    else:
        nextStation = np.nanmax(bicingStations["internalId"]) + 1
    for index, row in bicingStations.iterrows():
        internalId = row["internalId"]
        if np.isnan(internalId):
            internalId = nextStation
            UpdateStationsFile(internalId, row, stationsFile)
            CreateDataFile(internalId, outputPath)
            nextStation = nextStation + 1
        UpdateDataFile(internalId, row, outputPath)
    print("Bicing data updated at " + row["updateTime"])

In [ ]:
while True:
    SaveBicingData()
    time.sleep(SECONDS_UPDATE)